<a target="_blank" href="https://colab.research.google.com/github/Xornotor/PPGEE-Otimizacao-Exercicios/blob/main/Lista-01-A/Q7.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Lista de Exercícios 01-A | Questão 7**

**UFBA** | PPGEE0016 - Otimização

**Aluno:** André Paiva Conrado Rodrigues

In [1]:
# Importação de dependências e configurações de pretty-printing
import numpy as np
from scipy.optimize import linprog
np.set_printoptions(precision=3, suppress=True, linewidth=100)

## **1. Problema de Otimização**

Considerando:

*   $x_1$ A quantidade de carne de boi a ser carregada;
*   $x_2$ A quantidade de grãos a ser carregada;

Temos o seguinte problema de otimização:

\begin{equation*}
\begin{aligned}
\text{Maximizar} \\
&& Z = 0.35x_1 + 0.12x_2 \\
\text{Sujeito a} \\
c_{1}: && x_1 + x_2 \leq 160000 \\
c_{2}: && 0.2x_1 + 0.4x_2 \leq 70000 \\
c_{3}: && x_1 \leq 85000 \\
c_{4}: && x_2 \leq 100000 \\
&& x_1, x_2 \geq 0 \\
\end{aligned}
\end{equation*}

## **2. Solução manual por Simplex**

Primeiramente, é necessário colocar a função objetivo e as restrições na forma padrão, adicionando variáveis de folga para eliminar as desigualdades. Deste modo, o problema é reformulado da seguinte maneira:

\begin{equation*}
\begin{aligned}
\text{Maximizar} \\
&& Z - 0.35x_1 - 0.12x_2 = 0 \\
\text{Sujeito a} \\
c_{1}: && x_1 + x_2 + x_3 = 160000 \\
c_{2}: && 0.2x_1 + 0.4x_2 + x_4 = 70000 \\
c_{3}: && x_1 + x_5 = 85000 \\
c_{4}: && x_2 + x_6 = 100000 \\
&& x_1, x_2, x_3, x_4, x_5, x_6 \geq 0 \\
\end{aligned}
\end{equation*}

Montando a matriz $\boldsymbol{M}$ para o algoritmo Simplex:

|        ////        | $Z$ | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $x_6$ |  $b$   |
|--------------------|-----|-------|-------|-------|-------|-------|-------|--------|
| $\boldsymbol{L_1}$ |  0  |   1   |   1   |   1   |   0   |   0   |   0   | 160000 |
| $\boldsymbol{L_2}$ |  0  |  0.2  |  0.4  |   0   |   1   |   0   |   0   |  70000 |
| $\boldsymbol{L_3}$ |  0  |   1   |   0   |   0   |   0   |   1   |   0   |  85000 |
| $\boldsymbol{L_4}$ |  0  |   0   |   1   |   0   |   0   |   0   |   1   | 100000 |
| $\boldsymbol{L_5}$ |  1  | -0.35 | -0.12 |   0   |   0   |   0   |   0   |   0    |

In [2]:
M = np.array([
    [0,  1,     1,     1,  0,  0,  0,  160000 ],
    [0,  0.2,   0.4,   0,  1,  0,  0,  70000  ],
    [0,  1,     0,     0,  0,  1,  0,  85000  ],
    [0,  0,     1,     0,  0,  0,  1,  100000 ],
    [1, -0.35, -0.12,  0,  0,  0,  0,  0      ]
], dtype=np.float64)

### **Iteração 1**

Iniciamos selecionando a coluna de $x_1$ como **coluna pivô** por possuir custo com menor valor (-5). A partir disto, efetuamos a divisão *element-wise* dos elementos da coluna $b$ pelos elementos da coluna $x_1$, e selecionando o menor valor, obtemos a **linha pivô**.

In [3]:
b_col = 7
pivot_col = 1
div_pivot_b_cols = (M.T[b_col] / M.T[pivot_col])[:-1]
div_pivot_b_cols

/tmp/ipykernel_149903/992770500.py:3: RuntimeWarning: divide by zero encountered in divide
  div_pivot_b_cols = (M.T[b_col] / M.T[pivot_col])[:-1]


array([160000., 350000.,  85000.,     inf])

In [4]:
pivot_row = np.argmin(div_pivot_b_cols)
pivot_row

np.int64(2)

Logo, a linha pivô é $L_3$. Não é necessário fazer nenhuma substituição, já que $M_{32} = 1$. A ideia agora é zerar todos os outros elementos da coluna $x_1$. Para tanto, fazemos as seguintes substituições:

* $L_1 \rightarrow L_1 - L_3$
* $L_2 \rightarrow L_2 - 0.2 L_3$
* $L_5 \rightarrow L_5 + 0.35 L_3$

In [5]:
for i in range(M.shape[0]):
    if(i != pivot_row):
        M[i] -= M[i][pivot_col] * M[pivot_row]
M

array([[     0.  ,      0.  ,      1.  ,      1.  ,      0.  ,     -1.  ,      0.  ,  75000.  ],
       [     0.  ,      0.  ,      0.4 ,      0.  ,      1.  ,     -0.2 ,      0.  ,  53000.  ],
       [     0.  ,      1.  ,      0.  ,      0.  ,      0.  ,      1.  ,      0.  ,  85000.  ],
       [     0.  ,      0.  ,      1.  ,      0.  ,      0.  ,      0.  ,      1.  , 100000.  ],
       [     1.  ,      0.  ,     -0.12,      0.  ,      0.  ,      0.35,      0.  ,  29750.  ]])

### **Iteração 2**

Agora, a coluna pivô é a coluna de $x_2$. Efetuamos a divisão *element-wise* entre a coluna $b$ e a coluna $x_2$ para obter a linha pivô:

In [6]:
pivot_col = 2
div_pivot_b_cols = (M.T[b_col] / M.T[pivot_col])[:-1]
div_pivot_b_cols

/tmp/ipykernel_149903/1391391574.py:2: RuntimeWarning: divide by zero encountered in divide
  div_pivot_b_cols = (M.T[b_col] / M.T[pivot_col])[:-1]


array([ 75000., 132500.,     inf, 100000.])

In [7]:
pivot_row = np.argmin(div_pivot_b_cols)
pivot_row

np.int64(0)

A linha pivô agora é $L_1$. Não é necessário fazer substituições, já que $M_{13} = 1$. Para zerar os outros elementos da coluna $x_2$:

* $L_2 \rightarrow L_2 - 0.4 L_1$
* $L_4 \rightarrow L_4 - L_1$
* $L_5 \rightarrow L_5 + 0.12 L_1$

In [8]:
for i in range(M.shape[0]):
    if(i != pivot_row):
        M[i] -= M[i][pivot_col] * M[pivot_row]
M

array([[    0.  ,     0.  ,     1.  ,     1.  ,     0.  ,    -1.  ,     0.  , 75000.  ],
       [    0.  ,     0.  ,     0.  ,    -0.4 ,     1.  ,     0.2 ,     0.  , 23000.  ],
       [    0.  ,     1.  ,     0.  ,     0.  ,     0.  ,     1.  ,     0.  , 85000.  ],
       [    0.  ,     0.  ,     0.  ,    -1.  ,     0.  ,     1.  ,     1.  , 25000.  ],
       [    1.  ,     0.  ,     0.  ,     0.12,     0.  ,     0.23,     0.  , 38750.  ]])

Repare que a linha de coeficientes de custo agora tem apenas valores positivos! Isto significa que chegamos ao final das iterações.

### **Resultado final**

Podemos ver, pelo resultado final, que ambas as variáveis originais fazem parte da base, e que a resposta final é:

* $x_1 = 85000$;
* $x_2 = 75000$;

O lucro máximo é de $\textrm{R\$ } 38750$.

## **3. Comprovação da solução por algoritmo**

In [9]:
A = np.array([[1, 1], [0.2, 0.4]])

b = [160000, 70000]

coef = np.array([0.35, 0.12]) * (-1)

var_bounds = np.array([[0, 0], [85000, 100000]]).T

res = linprog(coef, A_ub=A, b_ub=b, bounds=var_bounds, method='highs')

In [10]:
print(f"Valor máximo da função objetivo: {-res.fun}")
print(f"x1 = {res.x[0]}, x2 = {res.x[1]}")

Valor máximo da função objetivo: 38750.0
x1 = 85000.0, x2 = 75000.0
